In [1]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import hog
import glob

In [2]:
class SimpleDatasetLoader:
    def __init__(self, preprocessors=None):
        self.preprocessors = preprocessors if preprocessors else []
    
    def load(self, image_paths, verbose=1):
        data, labels = [], []
        class_names = sorted(os.listdir('Helmet_Dataset'))
        
        for class_idx, class_name in enumerate(class_names):
            class_path = os.path.join('Helmet_Dataset', class_name)
            image_paths = glob.glob(os.path.join(class_path, "*.jpg")) + \
                         glob.glob(os.path.join(class_path, "*.png")) + \
                         glob.glob(os.path.join(class_path, "*.jpeg"))
            
            for img_path in image_paths:
                image = cv2.imread(img_path)
                if image is not None:
                    image = cv2.resize(image, (32, 32))
                    image = image.astype("float") / 255.0
                    data.append(image.flatten())
                    labels.append(class_idx)
        
        return np.array(data), np.array(labels), class_names


In [3]:
# Load dataset
print("Loading dataset...")
dl = SimpleDatasetLoader()
(data, labels, class_names) = dl.load('Helmet_Dataset')

print(f"Loaded {len(data)} images")
print(f"Classes: {class_names}")

# Split data
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.25, random_state=42)



Loading dataset...
Loaded 671 images
Classes: ['Helmet', 'Person_no_helmet', 'no_person']


In [4]:
# HOG Feature Extractor
def extract_hog_features(images):
    hog_features = []
    for image in images:
        image_reshaped = image.reshape(32, 32, 3)
        gray = cv2.cvtColor((image_reshaped * 255).astype(np.uint8), cv2.COLOR_BGR2GRAY)
        feature = hog(gray, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(2, 2), 
                      block_norm='L2-Hys', visualize=False)
        hog_features.append(feature)
    return np.array(hog_features)

# Extract HOG features
print("Extracting HOG features...")
trainX_hog = extract_hog_features(trainX)
testX_hog = extract_hog_features(testX)

# Create and train pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),  # Keep 95% variance
    ('svm', SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42))
])

print("Training SVM classifier...")
pipeline.fit(trainX_hog, trainY)



Extracting HOG features...
Training SVM classifier...


,steps,"[('scaler', ...), ('pca', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,n_components,0.95
,copy,True
,whiten,False
,svd_solver,'auto'


In [5]:
# Predictions
print("Making predictions...")
train_predictions = pipeline.predict(trainX_hog)
test_predictions = pipeline.predict(testX_hog)


Making predictions...


In [ ]:
# Results
print("\nTRAINING RESULTS:")
print(classification_report(trainY, train_predictions, target_names=class_names))
print(f"Training Accuracy: {accuracy_score(trainY, train_predictions):.4f}")

print("\nTESTING RESULTS:")
print(classification_report(testY, test_predictions, target_names=class_names))
print(f"Test Accuracy: {accuracy_score(testY, test_predictions):.4f}")

# Save model
import joblib
joblib.dump(pipeline, 'Models/helmet_classifier.pkl')
print("\nModel saved as 'helmet_classifier.pkl'")


TRAINING RESULTS:
                  precision    recall  f1-score   support

          Helmet       0.99      1.00      1.00       207
Person_no_helmet       1.00      0.99      1.00       172
       no_person       1.00      0.99      1.00       124

        accuracy                           1.00       503
       macro avg       1.00      1.00      1.00       503
    weighted avg       1.00      1.00      1.00       503

Training Accuracy: 0.9960

TESTING RESULTS:
                  precision    recall  f1-score   support

          Helmet       0.65      0.95      0.77        73
Person_no_helmet       0.70      0.56      0.62        50
       no_person       0.91      0.44      0.60        45

        accuracy                           0.70       168
       macro avg       0.75      0.65      0.66       168
    weighted avg       0.73      0.70      0.68       168

Test Accuracy: 0.6964

Model saved as 'helmet_classifier.pkl'


# Predict the single image

In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
import joblib

# Load saved model and class names
model = joblib.load('Models/helmet_classifier.pkl')
class_names = ['Helmet', 'no_person', 'Person_no_helmet']  # keep the same order as training

# HOG feature extraction function for a single image
def extract_hog_single(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    features = hog(gray, orientations=8, pixels_per_cell=(4, 4), cells_per_block=(2, 2),
                   block_norm='L2-Hys', visualize=False)
    return features

# Load image to predict
image_path = 'Test_Files/Person_with_helmet.jpg'  # replace with your image path
image = cv2.imread(image_path)
if image is None:
    raise Exception("Image not found or error reading image")

# Preprocess and extract features
image_resized = cv2.resize(image, (32, 32))
hog_features = extract_hog_single(image_resized)

# Predict class
predicted_class_index = model.predict([hog_features])[0]
predicted_class_name = class_names[predicted_class_index]

print(f"Predicted class: {predicted_class_name}")


Predicted class: no_person
